## Tool Calling with the Converse API and Amazon Nova via SAP GenAI Hub

This Python notebook demonstrates how to implement tool calling using the Converse API, enabling agentic workflows with SAP GenAI Hub and Amazon Nova models. Below is a step-by-step explanation of the process:

First we import the Bedrock client drop in replacement for the SAP GenAI Hub SDK

In [ ]:
from gen_ai_hub.proxy.native.amazon.clients import Session

# Initialize the Bedrock client via SAP GenAI Hub
bedrock = Session().client(model_name="amazon--nova-lite")
# bedrock = Session().client(model_name="anthropic--claude-3-haiku")

We'll create a basic weather tool that will generate a random temperature for the city passed in

The tools available to the model can be passed through the toolConfig parameter of the converse API. It is a best practice to be specific and consise. The schema will help the model determine when and how to use the tools available

In [2]:
import string
import random


def get_weather(city: string):
    return f"{random.randint(-5, 45)} degrees Celcius"


tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "getWeather",  # Name of the tool
                "description": "A helpful weather tool to retrieve the current weather based on a city",  # Helpful description of the tool's functionality
                "inputSchema": {
                    "json": {
                        "type": "object",  
                        "properties": {
                            "city": {  # The name of the parameter
                                "type": "string",  # parameter type: string/int/etc
                                "description": "The city to retrieve the weather for.",  # Helpful description of the parameter
                            }
                        },
                        "required": ["city"],  # List of all required parameters
                    }
                },
            }
        }
    ]
}

Now that we've defined our tool, we'll put together some code that will call the model and invoke the tool if it is called.

Once the tool is called we pass it back to the model as a toolResult content type
```python
{ "role": "user", "content": [{ "toolResult": { "toolUseId": tool_use["toolUseId"], "content": [{ "json": { "weather": weather }}]}}]}
```

In [5]:
import json

def nova_travel_guide(city: string):
    print("-----------------")
    user_message = f"Can you recommend some activities for me to do in {city} based on the current weather?"
    print(f"User Message: {user_message}")

    system_list = [
        {
            "text": "You are a helpful travel assistant. You have access to external tools to help give recommendations to the user"
        }
    ]

    # Define one or more messages using the "user" and "assistant" roles.
    message_list = [
        {
            "role": "user", 
            "content": [
                {
                    "text": user_message
                }
            ]
        },
    ]

    inf_params = {"topP": 1, "temperature": 1}

    initial_response = bedrock.converse(
        system=system_list,
        messages=message_list,
        inferenceConfig=inf_params,
        # additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
        toolConfig=tool_config,
    )
    print("\n[Initial Response]")
    print(f"Stop Reason: {initial_response['stopReason']}")
    print(f"Content: {json.dumps(initial_response['output']['message'], indent=2)}")

    if initial_response["stopReason"] == "tool_use":
        tool_use = next(
            block["toolUse"]
            for block in initial_response["output"]["message"]["content"]
            if "toolUse" in block
        )

        if tool_use["name"] == "getWeather":

            print(f"\nTool Name: {tool_use['name']}")
            print(f"Tool Input: {tool_use['input']}")

            weather = get_weather(tool_use["input"]["city"])

            print(f"Tool Result: {weather}")

            final_messages = [
                *message_list,
                initial_response["output"]["message"],
                {
                    "role": "user",
                    "content": [
                        {
                            "toolResult": {
                                "toolUseId": tool_use["toolUseId"],
                                "content": [{"json": {"weather": weather}}],
                            }
                        }
                    ],
                },
            ]
            final_response = bedrock.converse(
                messages=final_messages,
                inferenceConfig=inf_params,
                # additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
                toolConfig=tool_config,
            )

            output = next(
                block["text"]
                for block in final_response["output"]["message"]["content"]
                if "text" in block
            )
            print(f"\nResponse: {output}")
        else:
            print("The weather tool was not called")

Now lets interact with the tool!

In [6]:
nova_travel_guide("Munich")
nova_travel_guide("San Francisco")

-----------------
User Message: Can you recommend some activities for me to do in Munich based on the current weather?

[Initial Response]
Stop Reason: tool_use
Content: {
  "role": "assistant",
  "content": [
    {
      "text": "Okay, let's see what activities I can recommend for Munich based on the current weather. "
    },
    {
      "toolUse": {
        "toolUseId": "tooluse_16Cdg5LJTNePrEGI89N2Mg",
        "name": "getWeather",
        "input": {
          "city": "Munich"
        }
      }
    }
  ]
}

Tool Name: getWeather
Tool Input: {'city': 'Munich'}
Tool Result: 24 degrees Celcius

Response: Given that the current temperature in Munich is 24 degrees Celsius, here are some recommended activities:

- Visit one of Munich's beautiful parks like the English Garden or Olympiapark and go for a relaxing stroll or have a picnic.
- Explore the historic center of Munich, including Marienplatz, the Alte Pinakothek art museum, and the Frauenkirche cathedral.
- Go to an outdoor beer gar